# Part 2: Word Embeddings

In this part of the assignment, we'll explore word embeddings a little more deeply.

If you haven't seen the [week3/embeddings.ipynb](../../../materials/week3/embeddings.ipynb) demo notebook, we recommend you look through it; this part of the assignment will build on that material.

In [1]:
# This runs a shell command from the notebook.
!pip install plotly

# Standard python helper libraries.
import os, sys, re, json, time
import itertools, collections
from IPython.display import display

# NumPy and SciPy for matrix ops
import numpy as np
import scipy.sparse

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# NLTK for NLP utils
import nltk

# Helper libraries
from shared_lib import utils, vocabulary, tf_embed_viz
import part2_helpers

# Plotly imports.
import plotly.offline as plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go

## Part (a): Nearest Neighbors

In this part, we'll explore nearby words to get a feel for how our embeddings capture word similarity.

We'll re-use the [week3/embeddings.ipynb](../../../materials/week3/embeddings.ipynb) code to build SVD-based embeddings from word-word co-occurrence counts. To keep this notebook brief, most of the code is included in `part2_helpers.py`, but we've glued it all together in the `embeddings_from_corpus` function below:

- Load the corpus, process to tokens, and convert to ids.
- Compute a word-word co-occurrence matrix $C_{ij}$ with a window of size $\pm K$.
- Apply the PPMI transformation to $C$.
- Compute the SVD of $C$, truncated to $d$ dimensions.

The end result is an embedding matrix $W$ of shape $|V| \times d$, where each row $W_i$ is the $d$-dimensional representation of word $i$.

In [2]:
def embeddings_from_corpus(corpus_name, K=1, d=100):
    if corpus_name == "reuters":
        assert(nltk.download('punkt'))
    ##
    # Load and pre-process the corpus
    assert(nltk.download(corpus_name))  # make sure we have the data
    corpus = utils.get_corpus(corpus_name)
    vocab = vocabulary.Vocabulary(utils.canonicalize_word(w) for w in utils.flatten(corpus.sents()))
    print "Vocabulary: %d words" % vocab.size

    tokens = part2_helpers.sents_to_tokens(corpus.sents(), vocab)
    print "%g tokens" % len(tokens)
    token_ids = vocab.words_to_ids(tokens)
    
    ##
    # Compute co-occurrence matrix and word vectors
    t0 = time.time()
    C = part2_helpers.cooccurrence_matrix(token_ids, vocab.size, K=K)
    print "Computed Co-occurrence matrix in %s" % utils.pretty_timedelta(since=t0); t0 = time.time()
    C_ppmi = part2_helpers.PPMI(C)
    print "Computed PPMI in %s" % utils.pretty_timedelta(since=t0); t0 = time.time()
    Wv, _ = part2_helpers.SVD(C_ppmi, d=d)
    print "Computed SVD in %s" % utils.pretty_timedelta(since=t0)
    
    return Wv, vocab

Use our code to build embeddings on the Brown corpus:

In [3]:
Wv1_brown, vocab_brown = embeddings_from_corpus('brown', K=1, d=100)

NLTK includes wrappers for a number of corpora; the full list is available at http://www.nltk.org/nltk_data/

For this assignment, we'll be using:
- [The Brown corpus](http://www.hit.uib.no/icame/brown/bcm.html): `'brown'`
- [The Reuters corpus](http://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html): `'reuters'`
- [Movie review data](http://www.cs.cornell.edu/people/pabo/movie-review-data/): `'movie_reviews'`

These corpora are all similar in size, between 1-2 million words.

For completeness' sake, we've also included the TensorBoard visualization code from the demo. This is totally optional - you don't need it at all for this section.

In [4]:
# Change these to visualize a different set
Wv = Wv1_brown
vocab = vocab_brown

ev = tf_embed_viz.TFEmbeddingVizWrapper()
ev.write_vocab_file(words=vocab.ids_to_words(range(Wv.shape[0])))
ev.write_embeddings(Wv)

### Cosine Similarity

To measure the similarity of two words, we'll use the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) between their representation vectors:

$$ D^{cos}_{ij} = \frac{v_i^T v_j}{||v_i||\ ||v_j||}$$

*Note that this is called cosine similarity because $D^{cos}_{ij} = \cos(\theta_{ij})$, where $\theta_{ij}$ is the angle between the two vectors.*

### Part (a) tasks:

For some of these tasks, you'll want to define new variables and/or make new code cells as intermediate steps. For grading, however, we're only going to look at the designated cells.

1. Implement the `find_nn_cos` function in the cell below. Read the docstring _carefully_ - it describes what you should return. *Hint:* try to use NumPy functions (see below) instead of a `for` loop.
2. Using the `embedding_from_corpus` function, generate 100-dimensional embeddings from the Brown corpus with $K = 1$, $K = 3$, and $K = 5$. Using the `show_nns` function, find the nearest neighbors for the word `"washington"` in each set of embeddings. *Briefly* describe how the set of nearest neighbors changes as you increase the window width $K$. Inlcude the nearest-neighbor lists in your answer.
3. Generate 100-dimensional embeddings from the Reuters corpus (`'reuters'`) with $K = 3$. Find the nearest neighbors of the word `"money"` in these embeddings and in the $K=3$ embeddings from the Brown corpus. How are the sets of words different? Give a *brief* rationale for this based on your answer to the previous question and the differences between the corpora.
4. Generate 100-dimensional embeddings from the Movie reviews corpus (`'movie_reviews'`) with $K = 3$. Compare the nearest neighbors of the word `"great"` against the $K = 3$ embeddings from Brown and Reuters.

### Answers to Part (a)

**Put your answers to 2, 3, and 4 here.**

1. Fill in code cell below.
2. *Your answer here!*
3. *Your answer here!*
4. *Your answer here!*

In [5]:
## GRADED_CELL: part_a_1
def find_nn_cos(v, Wv, k=10):
    """Find nearest neighbors of a given word, by cosine similarity.
    
    Returns two parallel lists: indices of nearest neighbors, and 
    their cosine similarities. Both lists are in descending order, 
    and inclusive: so nns[0] should be the input word, nns[1] should be 
    the index of the first nearst neighbor, and so on.
    
    You may find the following numpy functions useful:
      np.linalg.norm : take the l2-norm of a vector or matrix
      np.dot : dot product or matrix multiplication
      np.argsort : get indices sorted by element value,
        so np.argsort(numbers)[-5:] will return the top five elements
    
    Args:
      v: (d-dimensional vector) word vector of interest
      Wv: (V x d matrix) word embeddings
      k: (int) number of neighbors to return
    
    Returns (nns, ds), where:
      nns: (k-element vector of ints), 
        row indices of nearest neighbors, including the given word
      ds: (k-element vector of floats), 
        cosine similarity of each neighbor in nns
    """
    pass
    #### YOUR CODE HERE ####



    #### END(YOUR CODE) ####
    
def show_nns(word, Wv, vocab, k=10):
    """Helper function to print neighbors of a given word."""
    word = utils.canonicalize_word(word, wordset=vocab.wordset)
    print "Nearest neighbors for \"%s\"" % word
    for i, d in zip(*find_nn_cos(Wv[vocab.word_to_id[word]], Wv, k)):
        w = vocab.id_to_word[i]
        print "%.03f : \"%s\"" % (d, w)
    print ""

In lieu of a whole separate test module for this function, we'll just give a sanity check here:

In [6]:
# Sanity check find_nn_cos
inputs = (Wv[vocab_brown.word_to_id["washington"]], Wv1_brown, 3)
expected_nns = vocab_brown.words_to_ids(["washington", "england", "georgia"])
expected_ds = [1.000, 0.714, 0.696]
nns, ds = find_nn_cos(*inputs)

ok = all(nns == expected_nns)
print "Neighbors match - ok." if ok else "Neighbors don't match expected!"

ok = np.allclose(ds, [1.000, 0.714, 0.696], rtol=3e-3)
print "Distances match - ok." if ok else "Distances don't match expected!"

Use the `show_nns` function to print the nearest neighbors for a given word. For example:

In [7]:
show_nns("washington", Wv1_brown, vocab_brown)

In [8]:
## Part (a) 2.

 

In [9]:
## Part (a) 3.

 

In [10]:
## Part (a) 4.

 

## Part (b): Bag-of-Vectors Classifier

In Week 4, we'll apply our continuous word representations to the language modeling task. But word embeddings are useful for many other tasks as well!

Here, we'll build a simple sentiment classifier using our SVD-based embeddings. The dataset is `movie_reviews`, aka [Polarity Dataset 2.0](http://www.cs.cornell.edu/people/pabo/movie-review-data/) from Cornell, which contains 1000 positive ($y = 1$) and 1000 negative ($y = 0$) movie reviews.

Recall the simple bag-of-words logistic regression, which we might use as a baseline:

$$\hat{y} = \sigma(\sum_{x_{j} \in x} W_j x_{j} + b) = \sigma(W_{bow}x_{bow} + b)$$

where $x_{j}$ is the number of times the word $w_j$ appears in the example (document) $x$, and our model parameters are a $|V|$-dimensional weight vector $W$ and a bias term $b$.

If we have pre-trained word embeddings, a simple extension of this is a **bag-of-vectors** model. Starting from the bag-of-words representation, we'll take the embedding vector of each word, then represent the example $x$ as the sum (or average) of the $d$-dimensional word vectors $U$:

$$ x_{bov} = \sum_{x_j \in x} U_j x_j $$ 

If we treat these as $d$-dimensional features, we can train our usual logistic regression model $\hat{y} = \sigma (W_{bov} x_{bov} + b) $.

Let's build a bag-of-vectors model, and see how the word embeddings can help us generalize. We'll train our embeddings - which are *unsupervised* - on the whole corpus, then train our classifier on only a subset of it. This simulates the common case where we have a small amount of labeled data, but access to a huge quantity of unlabeled text.

First, we'll start with the bag-of-words matrix:

In [11]:
corpus = nltk.corpus.movie_reviews
vocab = vocabulary.Vocabulary(utils.canonicalize_word(w) 
                              for w in utils.flatten(corpus.sents()))
print "Vocabulary: %d words" % vocab.size

pos = corpus.paras(categories='pos')
neg = corpus.paras(categories='neg')
y = np.array([1]*len(pos) + [0]*len(neg), dtype=int)
print "%d positive examples" % sum(y == 1)
print "%d negative examples" % sum(y == 0)

In [12]:
# For bag-of-words, we'll construct a sparse matrix X
# Don't worry about how this code block works.
ii, jj = [], []
for i, para in enumerate(itertools.chain(pos, neg)):
    tokens = part2_helpers.sents_to_tokens(para, vocab)
    token_ids = vocab.words_to_ids(tokens)
    ii.extend([i]*len(token_ids))  # row indices
    jj.extend(token_ids)           # column indices (words)

X_bow = scipy.sparse.csr_matrix((np.ones_like(ii), (ii, jj)), 
                                shape=[len(y),vocab.size])

print ("Bag-of-words matrix X_bow: %d x %d" % X_bow.shape)
print ("  %g nonzero elements" % X_bow.nnz)
print ("  %g total tokens" % X_bow.sum())
print ("Label vector Y: %d elements" % y.shape)

We've provided some helper code in `part2_helpers.py` to call Scikit-learn's logistic regression routines. It will run grid search with random cross-validation and return `(accuracy, accuracy_stdev)` on the left-out data:

In [13]:
reload(part2_helpers)
from part2_helpers import train_logistic_cv
score, score_std = train_logistic_cv(X_bow, y, verbose=True)

# You should get about 81% with the bag-of-words model
assert(0.80 < score); assert(score < 0.85)

### Part (b) tasks:

Per usual convention, the data matrix $X$ is the matrix where each row $x_i$ is the features for the $i^{th}$ train/test example.

1. Given $n$ training examples, our bag-of-words data matrix $X_{bow}$ will be a sparse matrix of shape $n \times |V|$. Write the shape of the bag-of-*vectors* data matrix $X_{bov}$, and give a matrix equation to compute $X_{bov}$ given the word embedding matrix $U$.
2. Given your answer above, does using the word embeddings in a linear classifier add any expressive power to the model?
3. In the cell below (under `#### YOUR CODE HERE ####`), compute $X_{bov}$. You should re-use the `embeddings_from_corpus` function from part (a) to compute 100-dimensional embeddings with $K=3$ on the `movie_reviews` corpus.
4. Use the `train_logistic_cv` function to compare the performance of the bag-of-words model against the bag-of-vectors model, for different amounts of training data. (Note that your embeddings from 3. should be trained on the *entire* corpus.)
5. Which model performs better in the small-dataset case? What if we use a larger fraction of the corpus? *Briefly* explain how the embeddings help your model, and how they can hold it back.

### Answers to Part (b):

Answer 1, 2, and 5 here. Please keep answers brief (1-2 lines).

Hint: You can use LaTeX to typeset math, e.g. `$ f(x) = x^2 $` will render as $ f(x) = x^2 $.

1. *Your answer here!*
2. *Your answer here!*
3. Fill in code cell below.
4. Fill in code cell below.
5. *Your answer here!*

In [ ]:
## GRADED_CELL: part_b_3
#### YOUR CODE HERE ####


#### END(YOUR CODE) ####
print ("Bag-of-vectors matrix X_bov: %d x %d" % X_bov.shape)

In [ ]:
## Use this cell for part_b_4
# Will take a few minutes to run everything
ns = np.array([10, 20, 50, 100, 200, 500, 1000])
bow_scores = []  # scores, stds with bag-of-words model
bov_scores = []  # scores, stds with bag-of-vectors model
for N in ns:
    print "Running with N=%d" % N
    #### YOUR CODE HERE ####
    
    
    #### END(YOUR CODE) ####
bow_scores = np.array(bow_scores)
bov_scores = np.array(bov_scores)

In [ ]:
# Use this to plot the scores of the two models
data = [go.Scatter(x = ns, y = bow_scores[:,0], name="Bag-of-Words",
                   mode='markers+lines', 
                   error_y=dict(type='data', array=bow_scores[:,1])),
        go.Scatter(x = ns, y = bov_scores[:,0], name="Bag-of-Vectors",
                   mode='markers+lines',
                   error_y=dict(type='data', array=bov_scores[:,1]))]
layout = go.Layout(title="Dev accuracy by training examples", xaxis=dict(type='log'))
fig = go.Figure(data=data, layout=layout)
plotly.iplot(fig)